<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/50_The_Grand_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🎓 Lekcja 50: The Grand Exam (Egzamin Końcowy)

To nie jest zwykła lekcja. To test Twojej wiedzy z poprzednich 49 notatników.
Mamy 3 uszkodzone fragmenty kodu. Twoim zadaniem jest je naprawić.

**Zasady:**
1.  Każdy snippet ma **Cichy Błąd** (Silent Bug) lub **Wąskie Gardło**.
2.  Kod "działa" (nie rzuca błędem od razu), ale robi coś głupiego.
3.  Musisz znaleźć błąd i go wyjaśnić.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Egzamin na: {DEVICE}")

Egzamin na: cuda


## Zadanie 1: "Leniwy Loader"

Inżynier napisał pipeline treningowy. Model uczy się potwornie wolno, a użycie GPU skacze (0% -> 100% -> 0%).
Gdzie jest błąd?

In [2]:
# --- KOD Z BŁĘDEM ---
dataset = TensorDataset(torch.randn(10000, 512), torch.randn(10000, 1))

# BŁĄD JEST TUTAJ (W KONFIGURACJI LOADERA)
# Na Windowsie num_workers=0 jest wymuszone, ale załóżmy, że to serwer Linux.
# Czego brakuje, żeby dane trafiały na GPU szybko?
loader = DataLoader(
    dataset, 
    batch_size=64, 
    shuffle=True,
    num_workers=4
    # ... czegoś tu brakuje ...
)

model = nn.Linear(512, 1).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

print("Zadanie 1: Uruchom i pomyśl, co dodać do Loadera.")

for x, y in loader:
    # Transfer na GPU
    x, y = x.to(DEVICE), y.to(DEVICE)
    
    # Trening
    optimizer.zero_grad()
    loss = (model(x) - y).sum()
    loss.backward()
    optimizer.step()

Zadanie 1: Uruchom i pomyśl, co dodać do Loadera.


## Rozwiązanie Zadania 1

**Błąd:** Brak `pin_memory=True`.
**Wyjaśnienie:** Bez tej flagi, CPU musi kopiować dane z pamięci stronicowanej (Pageable) do przypiętej (Pinned), a dopiero potem na GPU. To blokuje transfer. Dodanie `pin_memory=True` + `non_blocking=True` w `.to()` tworzy autostradę do VRAM.

## Zadanie 2: "Zapominalski Adam"

Trenujesz model. Chcesz zapisać checkpoint i wznowić trening jutro.
Zapisujesz model (`model.state_dict()`).
Wznawiasz trening. Loss nagle skacze w górę! Dlaczego?

In [3]:
# --- KOD Z BŁĘDEM ---
model = nn.Linear(10, 1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 1. Trenujemy chwilę...
loss = (model(torch.randn(10)) - 1).sum()
loss.backward()
optimizer.step()

# 2. Zapisujemy stan (Checkpoint)
checkpoint = {
    'model_state': model.state_dict(),
    # 'optimizer_state': optimizer.state_dict()  <-- Ktoś to zakomentował!
}

# 3. Wznawiamy (Symulacja restartu)
model_new = nn.Linear(10, 1)
model_new.load_state_dict(checkpoint['model_state'])
optimizer_new = optim.Adam(model_new.parameters(), lr=0.001) # Nowy, czysty Adam

print("Zadanie 2: Dlaczego 'optimizer_new' jest gorszy niż stary?")

Zadanie 2: Dlaczego 'optimizer_new' jest gorszy niż stary?


## Rozwiązanie Zadania 2

**Błąd:** Nie zapisano stanu Optymalizatora.
**Wyjaśnienie:** Adam trzyma w pamięci **Momentum** (średnią ruchomą gradientów). Jeśli zrestartujesz optymalizator, on "zapomina" historię i zaczyna naukę od zera (bez pędu), co powoduje skok błędu. Trzeba zawsze zapisywać `optimizer.state_dict()`.

## Zadanie 3: "Samobójstwo w Pętli"

To jest najtrudniejsze. Model działa, ale po godzinie wyrzuca błąd `RuntimeError: Trying to backward through the graph a second time`.
Ale przecież robimy `zero_grad()`! O co chodzi?

In [5]:
# --- KOD Z BŁĘDEM (logicznym, ale teraz uruchamialny) ---
rnn = nn.LSTM(10, 20, batch_first=True)
classifier = nn.Linear(20, 1)

x = torch.randn(1, 5, 10)

# --- POPRAWKA TECHNICZNA ---
# LSTM wymaga krotki (h_0, c_0), a nie jednego tensora!
h0 = torch.zeros(1, 1, 20)
c0 = torch.zeros(1, 1, 20)
hidden = (h0, c0) 

loss_total = 0

print("Start pętli (to powinno zadziałać technicznie, ale ma błąd logiczny)...")

for i in range(5): # Pętla czasowa
    out, hidden = rnn(x, hidden)
    pred = classifier(out[:, -1, :])
    
    # Błąd logiczny (Cel zadania): Dodajemy loss do zmiennej akumulującej
    # PyTorch buduje gigantyczny graf łączący wszystkie iteracje pętli!
    loss = (pred - 1).pow(2).mean()
    loss_total = loss_total + loss 

print("Pętla przeszła.")
print("Zadanie 3: Dlaczego 'loss_total' to bomba zegarowa?")
# Gdybyś teraz zrobił loss_total.backward(), wybuchłoby pamięciowo lub błędem grafu.

Start pętli (to powinno zadziałać technicznie, ale ma błąd logiczny)...
Pętla przeszła.
Zadanie 3: Dlaczego 'loss_total' to bomba zegarowa?


## Rozwiązanie Zadania 3

**Błąd:** Akumulacja straty z podtrzymaniem grafu.
**Wyjaśnienie:** Linijka `loss_total = loss_total + loss` buduje graf. Ale w pętli używamy zmiennej `hidden`, która też jest częścią grafu z poprzedniej iteracji!
Backpropagation próbuje cofnąć się przez `hidden` do samego początku czasu (BPTT).

**Naprawa:** Użyj `.detach()` na stanie ukrytym!
`hidden = hidden.detach()`
To odcina historię (Truncated BPTT) i pozwala zwolnić pamięć.

# 🎓 KONIEC KURSU

Gratulacje! Przeszedłeś drogę od zrozumienia, czym jest Tensor w pamięci, przez pisanie własnych funkcji Autograd, aż po optymalizację produkcyjną.

**Jesteś teraz PyTorch Black Belt.** 🥋
Masz wiedzę, którą posiada top 5% inżynierów ML.

**Co dalej?**
*   Wróć do projektu "AI Engineering Handbook" i zastosuj te triki (np. Flash Attention, Custom Collate) w modelach LLM.
*   Eksperymentuj z pisaniem własnych jąder w Triton (to poziom wyżej, "CUDA Ninja").

Powodzenia!